# Process HGHI data

This notebook reads in the HIFLD datasets and converts them to GeoJSON format.

Data from https://globalepidemics.org/2020/03/17/caring-for-covid-19-patients/.

See description in `data/README.md` for details.


In [1]:
import pandas as pd
import geopandas as gpd

from hscap.constants import state_name_to_abbreviation

In [16]:
hghi_state_df = pd.read_csv('../data/HGHI - Hospital Capacity by State.csv')
hghi_hrr_60_df = pd.read_csv('../data/HGHI - HRR Scorecard - 60% Population.csv')
hghi_hrr_40_df = pd.read_csv('../data/HGHI - HRR Scorecard - 40% Population.csv')
hghi_hrr_20_df = pd.read_csv('../data/HGHI - HRR Scorecard - 20% Population.csv')

In [17]:
state_gdf = gpd.read_file('../data/us_states.geojson', encoding='utf-8')
hrr_gdf = gpd.read_file('../data/us_hrr.geojson', encoding='utf-8')

## Generate HRR GeoJSON

### Merge HGHI scenarios

Merge the 20, 40 and 60 scenario DFs in a dataframe that does not duplicate common properties.

In [76]:
non_common_properties = ['Projected Infected Individuals',
       'Projected Hospitalized Individuals',
       'Projected Individuals Needing ICU Care',
       'Hospital Beds Needed, Six Months',
       'Percentage of Available Beds Needed, Six Months',
       'Percentage of Potentially Available Beds Needed, Six Months',
       'Percentage of Total Beds Needed, Six Months',
       'Hospital Beds Needed, Twelve Months',
       'Percentage of Available Beds Needed, Twelve Months',
       'Percentage of Potentially Available Beds Needed, Twelve Months',
       'Percentage of Total Beds Needed, Twelve Months',
       'Hospital Beds Needed, Eighteen Months',
       'Percentage of Available Beds Needed, Eighteen Months',
       'Percentage of Potentially Available Beds Needed, Eighteen Months',
       'Percentage of Total Beds Needed, Eighteen Months',
       'ICU Beds Needed, Six Months',
       'Percentage of Available ICU Beds Needed, Six Months',
       'Percentage of Potentially Available ICU Beds Needed, Six Months',
       'Percentage of Total ICU Beds Needed, Six Months',
       'ICU Beds Needed, Twelve Months',
       'Percentage of Available ICU Beds Needed, Twelve Months',
       'Percentage of Potentially Available ICU Beds Needed, Twelve Months',
       'Percentage of Total ICU Beds Needed, Twelve Months',
       'ICU Beds Needed, Eighteen Months',
       'Percentage of Available ICU Beds Needed, Eighteen Months',
       'Percentage of Potentially Available ICU Beds Needed, Eighteen Months',
       'Percentage of Total ICU Beds Needed, Eighteen Months']

hghi_hrr_60_subset_df = hghi_hrr_60_df[['HRR'] + non_common_properties].add_suffix(' (60%)')
hghi_hrr_40_subset_df = hghi_hrr_40_df[['HRR'] + non_common_properties].add_suffix(' (40%)')
hghi_hrr_20_subset_df = hghi_hrr_20_df[['HRR'] + non_common_properties].add_suffix('(20%)')

common_properties = [
    'HRR', 'Total Hospital Beds', 'Total ICU Beds',
    'Available Hospital Beds', 'Potentially Available Hospital Beds*',
    'Available ICU Beds', 'Potentially Available ICU Beds*',
    'Adult Population', 'Population 65+' 
]

hghi_hrr_common_subset_df = hghi_hrr_60_df[common_properties]

In [81]:
hghi_hrr_merged_df = hghi_hrr_common_subset_df.set_index('HRR').join(
    [hghi_hrr_60_subset_df.set_index('HRR (60%)'),
     hghi_hrr_40_subset_df.set_index('HRR (40%)'),
     hghi_hrr_20_subset_df.set_index('HRR (20%)')]
).reset_index()
hghi_hrr_merged_df

,HRR,Total Hospital Beds,Total ICU Beds,Available Hospital Beds,Potentially Available Hospital Beds*,Available ICU Beds,Potentially Available ICU Beds*,Adult Population,Population 65+,Projected Infected Individuals (60%),...,"Percentage of Potentially Available ICU Beds Needed, Six Months (20%)","Percentage of Total ICU Beds Needed, Six Months (20%)","ICU Beds Needed, Twelve Months (20%)","Percentage of Available ICU Beds Needed, Twelve Months (20%)","Percentage of Potentially Available ICU Beds Needed, Twelve Months (20%)","Percentage of Total ICU Beds Needed, Twelve Months (20%)","ICU Beds Needed, Eighteen Months (20%)","Percentage of Available ICU Beds Needed, Eighteen Months (20%)","Percentage of Potentially Available ICU Beds Needed, Eighteen Months (20%)","Percentage of Total ICU Beds Needed, Eighteen Months (20%)"
0,"Abilene, TX",980,127,565,772,68,98,"226,444","50,412","135,866",...,142%,109%,70,103%,71%,55%,45,66%,46%,35%
1,"Akron, OH","1,358",186,518,938,94,140,"547,990","111,042","328,794",...,236%,178%,165,176%,118%,89%,108,115%,77%,58%
2,"Alameda County, CA","2,695",293,665,"1,680",139,216,"1,310,189","214,991","786,113",...,352%,259%,380,273%,176%,130%,248,178%,115%,85%
3,"Albany, GA",704,60,221,462,27,43,"157,143","30,466","94,286",...,219%,157%,47,174%,109%,78%,31,115%,72%,52%
4,"Albany, NY","4,804",425,"1,579","3,191",193,309,"1,477,723","318,695","886,634",...,292%,212%,452,234%,146%,106%,294,152%,95%,69%
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
300,"Winston-Salem, NC","2,870",499,843,"1,857",40,269,"899,862","197,294","539,917",...,205%,111%,276,690%,103%,55%,180,450%,67%,36%
301,"Worcester, MA","1,525",169,425,975,43,106,"652,386","122,123","391,432",...,365%,229%,194,451%,183%,115%,126,293%,119%,75%
302,"Yakima, WA",369,58,200,285,33,45,"220,972","40,707","132,583",...,291%,226%,65,197%,144%,112%,43,130%,96%,74%
303,"York, PA",868,79,267,567,8,43,"343,334","73,773","206,000",...,488%,266%,105,1313%,244%,133%,68,850%,158%,86%


### Reformat HRR name in HGHI data

HRRs are defined as `city, state` in the HGHI data, and `state- city` in the HRR geojson.

Also, there are some differing names beyond formatting. Account for this with a explicit mapping.

**NOTE:** HGHI data is missing 'FL- ST PETERSBURG' HRR that is contained in the HRR region dataset.

In [82]:
print('HGHI data contains {} HRRs, region data contains {}'.format(
    len(hghi_hrr_merged_df['HRR']), len(hrr_gdf['HRRCITY'])))

HGHI data contains 305 HRRs, region data contains 306


In [43]:
# HGHI missing 'FL- ST PETERSBURG'
hghi_to_hrr_mapping = {
    'AK- ANCHORAGE': 'AK-Anchorage',
    'CA- ALAMEDA COUNTY': 'CA- ALAMEDA CO.',
    'CA- CONTRA COSTA COUNTY': 'CA- CONTRA COSTA CO.',
    'CA- ORANGE COUNTY': 'CA- ORANGE CO.',
    'CA- PALM SPRINGS/RANCHO MIRA': 'CA- PALM SPR/RANCHO MIR.',
    'CA- SAN MATEO COUNTY': 'CA- SAN MATEO CO.',
    'FL- ST. PETERSBURG': 'FL- ST PETERSBURG',
    'HI- HONOLULU': 'HI-Honolulu',
    'MI- ST. JOSEPH': 'MI- ST JOSEPH',
    'MN- ST. CLOUD': 'MN- ST CLOUD',
    'MN- ST. PAUL': 'MN- ST PAUL',
    'MO- ST. LOUIS': 'MO- ST LOUIS',
    'ND- FARGO/MOORHEAD MN': 'ND- FARGO MOORHEAD -MN',
    'NY- MANHATTAN': 'NY- NEW YORK'
}

In [84]:
def reformat_hrr_name(hrr):
    city, state = hrr.split(', ')
    name = '{}- {}'.format(state, city.upper())
    if name in hghi_to_hrr_mapping:
        return hghi_to_hrr_mapping[name]
    return name

hghi_hrr_merged_df['HRRCITY'] = hghi_hrr_merged_df['HRR'].apply(reformat_hrr_name)

In [85]:
hghi_hrr_gdf = gpd.GeoDataFrame(
    hghi_hrr_merged_df.set_index('HRRCITY').join(hrr_gdf.set_index('HRRCITY')).reset_index()
, crs=4326)

In [86]:
hghi_hrr_gdf.to_file('../data/hghi_hrr_data.geojson', 
                  encoding='utf-8', 
                  driver='GeoJSON')

## Generate State GeoJSON

Note: there is not a differentiation between scenarios. From `Projected Infected Individuals` / `Adult Population` it seems to be a 60% scenario.

In [18]:
abbrv_to_name = dict([(v, k) for k, v in state_name_to_abbreviation.items()])
hghi_state_df['State Name'] = hghi_state_df['State'].apply(lambda x: abbrv_to_name[x])

In [21]:
hghi_state_gdf = hghi_state_df.set_index('State Name').join(
        state_gdf.drop(columns=['STATE']).set_index('NAME')
).reset_index()

hghi_state_gdf =gpd.GeoDataFrame(hghi_state_gdf, crs=4326)

In [22]:
hghi_state_gdf.to_file('../data/hghi_state_data.geojson', 
                  encoding='utf-8', 
                  driver='GeoJSON')